## Fetch Locations

import libraries

In [1]:
import sys
sys.path.append("../src")
from api import foursquare
import pandas as pd
import json

Get coordinates of the Mercyhurst University (Erie, Pennsylvania).
This is where the survey took place.

In [9]:
#
with open("../data/raw/mercyhurst_coordinates.txt") as file:
    ll = json.loads(file.read())['ll']
    file.close()

Form a query to check for residential locations in a fixed radius around the Mercyhurst University.

In [3]:
fsc=foursquare.FoursqaurClient("../src/api/credentials.txt")
response = fsc.request_residentials(ll=ll, radius=100000, limit=50)
response

<Response [200]>

parse the response data (resedential locations) into a usable dataframe and clean the data.

In [4]:
results=response.json()['results']


In [10]:
print(f'there are {len(results)} results in the response')
#make a list of dictionaries with the fsq_id ,latitude, longitude, name, address, city, state, country and postal_code
venues=[]
for venue in results:
    venues.append({'latitude':venue['geocodes']['main']['latitude'],
                   'longitude':venue['geocodes']['main']['longitude'],
                   'fsq_id':venue['fsq_id'],
                   'address':venue['location']['address'],
                   'country':venue['location']['country'],
                   'postal_code':venue['location']['postcode'],
                   'region':venue['location']['region']})

there are 47 results in the response


In [11]:
venues_df=pd.DataFrame(venues)
venues_df.head()

,latitude,longitude,fsq_id,address,country,postal_code,region
0,42.123440,-80.077909,4f2293846d867182dff007d9,153 E 13th St,US,16503,PA
1,42.126495,-80.087438,4ef9be2cc512f2277a8b55f8,160 W 8th St,US,16501,PA
2,42.067218,-80.089730,4bf522b498ac0f47fbb564a8,1717 Kuntz Rd,US,16509,PA
3,42.091566,-79.296095,512bd37ae4b01d58196c800a,2109 Southwestern Dr,US,14750,NY
4,42.109142,-79.287287,5143ddc752cd4de5f8eecdec,75 Marine Park Dr,US,14720,NY


Form another query to get all nearby amenities (fixed in a short distance around each residential location).

In [12]:
n = len(venues_df)
restaurants = []
groceries = []
for i in range(n):
    venue = venues_df.iloc[i]
    ll = f"{venue['latitude']},{venue['longitude']}"
    response_restaurants = fsc.request_restaurants(ll=ll, radius=1000, limit=50)
    restaurants.append(len(response_restaurants.json()['results']))
    response_groceries = fsc.request_groceries(ll=ll, radius=1000, limit=50)
    groceries.append(len(response_groceries.json()['results']))



[41, 6, 49, 5, 26, 3, 11, 3, 1, 0, 0, 0, 0, 0, 7, 2, 31, 2, 28, 0, 24, 1, 26, 0, 26, 0, 11, 1, 4, 0, 10, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 16, 2, 7, 0, 0, 0, 2, 2, 1, 0, 7, 1, 16, 2, 4, 1, 6, 2, 14, 5, 5, 1, 19, 4, 24, 3, 14, 3, 2, 0, 0, 0, 20, 0, 1, 0, 0, 0, 7, 0, 5, 1, 0, 1, 0, 2, 0, 0, 6, 3, 19, 2]


In [24]:
print(restaurants)


[41, 49, 26, 11, 1, 0, 0, 7, 31, 28, 24, 26, 26, 11, 4, 10, 0, 0, 1, 0, 0, 16, 7, 0, 2, 1, 7, 16, 4, 6, 14, 5, 19, 24, 14, 2, 0, 20, 1, 0, 7, 5, 0, 0, 0, 6, 19]


In [23]:
print(groceries)


[6, 5, 3, 3, 0, 0, 0, 2, 2, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 1, 2, 1, 2, 5, 1, 4, 3, 3, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 3, 2]


In [27]:
venues_df['restaurants'] = restaurants
venues_df['groceries'] = groceries

In [1]:
print(venues_df[['restaurants','groceries']].head())

NameError: name 'venues_df' is not defined

In [ ]:
#save venues_df to a csv file
venues_df.to_csv('../data/processed/venues.csv', index=False)